# RAG over Business Reports Using Hybrid Retrieval

This notebook demonstrates how to combine sparse (keyword-based) and dense (embedding-based) retrieval to support a RAG-based system:
- Keywords from the `reports.csv`
- Descriptions from `Reporting_Inventory.xlsx`


<a href="https://colab.research.google.com/github/cbadenes/semantic-report-search/blob/main/data/analysis/42_rag.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>

In [1]:
!pip install -q sentence-transformers scikit-learn pandas
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.7 MB/s eta 0:00:00


In [2]:
# Revisar las primeras filas de la hoja "Views"
raw_inventory_df = pd.read_excel("Reporting_Inventory.xlsx", sheet_name="Views")
raw_inventory_df.head(2)


,ID Data Product,Report Name,Product Owner,PBIX_File,Report View,Description,Category,Status,Rename,Dimensions,KPIs,Other Terms,Filters,Tags,Priority
0,RPPBI0032,Feeder Market - 2024,Jonathan Shields,LifeReport.pbix,CRITERIA,Methodolody and definition of the algorithim o...,Informative,Productive,NaN,NaN,NaN,NaN,NaN,NaN,Priority 1
1,RPPBI0032,Feeder Market - 2024,Jonathan Shields,LifeReport.pbix,DESTINATION_OF_FEEDER_MARKETS,View focused on understand the performance by ...,Functional,Productive,NaN,"Hotel, month, Feeder Market, Segment, Channel ...","Total Revenue, Room Revenue, RN, Lead Time, Le...",NaN,NaN,NaN,Priority 1


Load and Merge Data

In [3]:
# Load source files
reports_df = pd.read_csv("reports.csv")
inventory_df = pd.read_excel("Reporting_Inventory.xlsx", sheet_name="Views")

# Merge by 'ID Data Product'
merged_df = reports_df.merge(inventory_df, on="ID Data Product", how="left")
merged_df = merged_df.rename(columns={"Report View_x": "Report View"})
merged_df = merged_df.drop(columns=["Report View_y"])
merged_df = merged_df.rename(columns={"Report Name_x": "Report Name"})
merged_df = merged_df.drop(columns=["Report Name_y"])
merged_df = merged_df.rename(columns={"Tags_x": "Tags"})
merged_df = merged_df.drop(columns=["Tags_y"])



# Clean fields
merged_df["keywords"] = merged_df["keywords"].fillna("")
merged_df["Description"] = merged_df["Description"].fillna("")
merged_df.head(2)

,ID Data Product,Report Name,Report View,Tags,keywords,Product Owner,PBIX_File,Description,Category,Status,Rename,Dimensions,KPIs,Other Terms,Filters,Priority
0,RPPBI0032,Feeder Market - 2024,CRITERIA,NaN,"2024, criterion, definition, feed, feeder mark...",Jonathan Shields,LifeReport.pbix,Methodolody and definition of the algorithim o...,Informative,Productive,NaN,NaN,NaN,NaN,NaN,Priority 1
1,RPPBI0032,Feeder Market - 2024,CRITERIA,NaN,"2024, criterion, definition, feed, feeder mark...",Jonathan Shields,LifeReport.pbix,View focused on understand the performance by ...,Functional,Productive,NaN,"Hotel, month, Feeder Market, Segment, Channel ...","Total Revenue, Room Revenue, RN, Lead Time, Le...",NaN,NaN,Priority 1


Prepare Sparse Representations (TF-IDF)

In [4]:
tfidf = TfidfVectorizer()
sparse_matrix = tfidf.fit_transform(merged_df["keywords"])


Prepare Dense Representations (Embeddings)

In [5]:
dense_model = SentenceTransformer("all-MiniLM-L6-v2")
dense_matrix = dense_model.encode(merged_df["Description"], convert_to_tensor=False)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Define Hybrid Search Function

In [6]:
def hybrid_search(query, alpha=0.5, top_k=5):
    # Sparse vector
    sparse_query = tfidf.transform([query])
    sparse_scores = cosine_similarity(sparse_query, sparse_matrix).flatten()

    # Dense vector
    dense_query = dense_model.encode([query])[0]
    dense_scores = cosine_similarity([dense_query], dense_matrix).flatten()

    # Combine scores
    hybrid_scores = alpha * sparse_scores + (1 - alpha) * dense_scores

    # Get top indices and scores
    top_indices = np.argsort(hybrid_scores)[::-1][:top_k]
    top_scores = hybrid_scores[top_indices]

    # Build result DataFrame
    results = merged_df.iloc[top_indices].copy()
    results["score"] = top_scores

    return results[["Report View", "keywords", "Description", "score"]]


Try It Out!

In [7]:
hybrid_search("market performance in European destinations")


,Report View,keywords,Description,score
14297,C4C_Qualification_Detail,"account, account handler, account segmentation...",View to analyze Key Potential Destinations by ...,0.442738
14299,Key Potential Destinations,"account (business travel, business travel & mo...",View to analyze Key Potential Destinations by ...,0.376392
25,EXECUTIVE VIEW,"2024, adr, aov, executive, feeder, feeder mark...",Benchmark by Destination. Outside information ...,0.363467
106,EXECUTIVE VIEW,"2025, adr, aov, executive, feeder, feeder mark...",Benchmark by Destination. Outside information ...,0.363279
19,EXECUTIVE VIEW,"2024, adr, aov, executive, feeder, feeder mark...",View focused on understand the performance by ...,0.354969


In [8]:
hybrid_search("staff efficiency and complaints resolution", alpha=0.3)

,Report View,keywords,Description,score
5899,Summary 24 vs 25,"commercial, efficiency, hide view, mastertool,...",View to measure commercial teams efficiency th...,0.371663
15304,Home Management,"2024, commercial, efficiency, home, index page...",Older version of the report that was launched ...,0.370699
5683,HOME,"commercial, efficiency, home, index page, inte...",View to measure commercial teams efficiency th...,0.363306
5701,Home Management,"commercial, efficiency, home, index page, inte...",View to measure commercial teams efficiency th...,0.360045
15312,Management View,"2024, commercial, commercial team, comparison,...",Older version of the report that was launched ...,0.351419


Generative Model

In [9]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# Load a local instruct model
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # or Qwen if preferred

tokenizer = AutoTokenizer.from_pretrained(model_id)
gen_model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")

rag_generator = pipeline(
    "text-generation",
    model=gen_model,
    tokenizer=tokenizer,
    return_full_text=False
)


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cuda:0


RAG function

In [10]:
def rag_hybrid_generation(question, alpha=0.5, top_k=5, max_tokens=200):
    # Sparse score
    sparse_vec = tfidf.transform([question])
    sparse_scores = cosine_similarity(sparse_vec, sparse_matrix).flatten()

    # Dense score
    dense_vec = dense_model.encode([question])[0]
    dense_scores = cosine_similarity([dense_vec], dense_matrix).flatten()

    # Combined hybrid score
    hybrid_scores = alpha * sparse_scores + (1 - alpha) * dense_scores
    top_indices = np.argsort(hybrid_scores)[::-1][:top_k]

    # Build retrieval context from both fields
    context = "\n\n".join(
        f"Report View: {merged_df.iloc[i]['Report View']}\n"
        f"- Keywords: {merged_df.iloc[i]['keywords']}\n"
        f"- Description: {merged_df.iloc[i]['Description']}"
        for i in top_indices
    )

    # Prompt for instruct model (TinyLlama-style)
    prompt = (
        "<|system|>\nYou are a helpful assistant that answers questions based on report metadata.\n"
        "<|user|>\n"
        f"Context:\n{context}\n\n"
        f"Question: {question}\n"
        "Answer:\n"
        "<|assistant|>\n"
    )

    # Generate answer
    response = rag_generator(prompt, max_new_tokens=max_tokens, do_sample=True)[0]["generated_text"]
    return response


In [11]:
rag_hybrid_generation("Which reports mention occupancy trends by destination?", alpha=0.4)


'The report "GOP Market Faces" and the report "Pricing vs Forward 90 Days" mention the trends in occupancy by destination.'